In [8]:
import csv

# Filepath for the CSV file (adjusted for Colab)
file_path = "/content/sample_books.csv"

# Data for the CSV file
books_data = [
    {"title": "1984", "author": "George Orwell", "year": 1949, "isbn": "1234567890", "genre": "Dystopian", "copies": 5},
    {"title": "To Kill a Mockingbird", "author": "Harper Lee", "year": 1960, "isbn": "1234567891", "genre": "Fiction", "copies": 3},
    {"title": "The Great Gatsby", "author": "F. Scott Fitzgerald", "year": 1925, "isbn": "1234567892", "genre": "Classic", "copies": 4},
    {"title": "Moby Dick", "author": "Herman Melville", "year": 1851, "isbn": "1234567893", "genre": "Adventure", "copies": 2},
    {"title": "Pride and Prejudice", "author": "Jane Austen", "year": 1813, "isbn": "1234567894", "genre": "Romance", "copies": 6},
]

# Writing the CSV file
with open(file_path, mode="w", newline="") as csvfile:
    fieldnames = ["title", "author", "year", "isbn", "genre", "copies"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Write headers
    writer.writeheader()

    # Write book data
    writer.writerows(books_data)

# Confirming the file is written
print(f"CSV file created at: {file_path}")


CSV file created at: /content/sample_books.csv


In [ ]:
import csv
from datetime import datetime
from datetime import timedelta
from collections import defaultdict

class Author:
    def __init__(self, name):
        self.name = name
        self.books = set()

    def add_book(self, book):
        self.books.add(book)


class Book:
    def __init__(self, title, author, year, isbn, genre, copies):
        self.title = title
        self.author = author
        self.year = year
        self.isbn = isbn
        self.genre = genre
        self.available_copies = copies
        self.waitlist = []

    def __str__(self):
        return f"{self.title} by {self.author.name} ({self.year}, {self.isbn}, {self.genre}, {self.available_copies}, {self.waitlist})"


class Customer:
    def __init__(self, name, customer_id):
        self.name = name
        self.customer_id = customer_id
        self.borrowed_books = {}

    def borrow_book(self, book):
        self.borrowed_books[book] = datetime.now()

    def return_book(self, book):
        self.borrowed_books.pop(book, None)

    def get_borrowed_books(self):
        return list(self.borrowed_books.keys())


class LibraryManagementSystem:
    def __init__(self):
        self.books = {}
        self.authors = defaultdict(Author)
        self.customers = {}
        self.genre_classification = defaultdict(list)
        self.customer_id_counter = 1

    def open_csv(self, file_name):
        try:
            with open(file_name, 'r') as file:
                reader = csv.DictReader(file)
                for row in reader:
                    self.add_book(
                        title=row['title'],
                        author_name=row['author'],
                        year=int(row['year']),
                        isbn=row['isbn'],
                        genre=row['genre'],
                        copies=int(row['copies'])
                    )
        except FileNotFoundError:
            print(f"File '{file_name}' not found.")
        except KeyError as e:
            print(f"Missing column in CSV: {e}")
        except ValueError as e:
            print(f"Invalid data in CSV: {e}")
    def add_book(self, title, author_name, year, isbn, genre, copies):
      # Check if the author already exists, otherwise create a new one
      if author_name not in self.authors:
          self.authors[author_name] = Author(author_name)
      author = self.authors[author_name]

      # Create and add the book
      book = Book(title, author, year, isbn, genre, copies)
      self.books[isbn] = book
      author.add_book(book)

      # Organize books by genre
      if genre not in self.genre_classification:
          self.genre_classification[genre] = []
      self.genre_classification[genre].append(book)


    def _add_book_to_genre(self, book):
        self.genre_classification[book.genre].append(book)

    def register_customer(self, name):
        customer_id = self.customer_id_counter
        self.customers[customer_id] = Customer(name, customer_id)
        self.customer_id_counter += 1
        return customer_id

    def borrow_book(self, customer_id, isbn):
        book = self.books.get(isbn)
        customer = self.customers.get(customer_id)

        if not book:
            print("Book not found.")
            return

        if not customer:
            print("Customer not found.")
            return

        if book.available_copies > 0:
            book.available_copies -= 1
            customer.borrow_book(book)
        else:
            print("Book unavailable. Adding to waitlist.")
            book.waitlist.append(customer)

    def return_book(self, customer_id, isbn):
        book = self.books.get(isbn)
        customer = self.customers.get(customer_id)

        if not book or not customer or book not in customer.borrowed_books:
            print("Invalid return operation.")
            return

        book.available_copies += 1
        customer.return_book(book)

        if book.waitlist:
            next_customer = book.waitlist.pop(0)
            print(f"Book assigned to {next_customer.name} from waitlist.")
            self.borrow_book(next_customer.customer_id, isbn)

    def search_books(self, query):
        query = query.lower()
        return [book for book in self.books.values()
                if query in book.title.lower() or
                query in book.author.name.lower() or
                query == book.isbn]

    def display_available_books(self):
        return [book for book in self.books.values() if book.available_copies > 0]

    def display_customer_books(self, customer_id):
        customer = self.customers.get(customer_id)
        return customer.get_borrowed_books() if customer else []

    def recommend_books(self, customer_id):
        customer = self.customers.get(customer_id)
        if not customer:
            print("Customer not found.")
            return []

        borrowed_genres = {book.genre for book in customer.get_borrowed_books()}
        recommendations = {book for genre in borrowed_genres
                           for book in self.genre_classification[genre]
                           if book not in customer.get_borrowed_books()}
        return list(recommendations)[:5]

    def check_late_returns(self, days):
        late_books = []
        threshold = datetime.now() - timedelta(days=days)
        for customer in self.customers.values():
            for book, borrow_date in customer.borrowed_books.items():
                if borrow_date < threshold:
                    late_books.append((customer.name, book))
        return late_books

    def run(self):
        while True:
            print("\nLibrary Management System")
            print("1. Load books from CSV")
            print("2. Add a book")
            print("3. Register a customer")
            print("4. Borrow a book")
            print("5. Return a book")
            print("6. Search for books")
            print("7. Display available books")
            print("8. Display customer borrowed books")
            print("9. Recommend books for a customer")
            print("10. Check late returns")
            print("11. Exit")

            choice = input("Enter your choice: ").strip()
            try:
                if choice == "1":
                    file_name = input("Enter CSV file name: ").strip()
                    self.open_csv(file_name)
                elif choice == "2":
                    title = input("Enter book title: ").strip()
                    author = input("Enter author name: ").strip()
                    year = int(input("Enter publication year: ").strip())
                    isbn = input("Enter ISBN: ").strip()
                    genre = input("Enter genre: ").strip()
                    copies = int(input("Enter number of copies: ").strip())
                    self.add_book(title, author, year, isbn, genre, copies)
                elif choice == "3":
                    name = input("Enter customer name: ").strip()
                    customer_id = self.register_customer(name)
                    print(f"Customer registered with ID: {customer_id}")
                elif choice == "4":
                    customer_id = int(input("Enter customer ID: ").strip())
                    isbn = input("Enter ISBN of the book to borrow: ").strip()
                    self.borrow_book(customer_id, isbn)
                elif choice == "5":
                    customer_id = int(input("Enter customer ID: ").strip())
                    isbn = input("Enter ISBN of the book to return: ").strip()
                    self.return_book(customer_id, isbn)
                elif choice == "6":
                    query = input("Enter search query (title/author/ISBN): ").strip()
                    results = self.search_books(query)
                    for book in results:
                        print(book)
                elif choice == "7":
                    books = self.display_available_books()
                    for book in books:
                        print(book)
                elif choice == "8":
                    customer_id = int(input("Enter customer ID: ").strip())
                    books = self.display_customer_books(customer_id)
                    for book in books:
                        print(book)
                elif choice == "9":
                    customer_id = int(input("Enter customer ID: ").strip())
                    recommendations = self.recommend_books(customer_id)
                    for book in recommendations:
                        print(book)
                elif choice == "10":
                    days = int(input("Enter number of days for late returns: ").strip())
                    late_books = self.check_late_returns(days)
                    for customer_name, book in late_books:
                        print(f"{customer_name} has a late return: {book}")
                elif choice == "11":
                    print("Exiting the system. Goodbye!")
                    break
                else:
                    print("Invalid choice. Please try again.")
            except Exception as e:
                print(f"An error occurred: {e}")

# Example usage:
if __name__ == "__main__":
    system = LibraryManagementSystem()
    system.run()  # Uncomment to enable interactive menu.



Library Management System
1. Load books from CSV
2. Add a book
3. Register a customer
4. Borrow a book
5. Return a book
6. Search for books
7. Display available books
8. Display customer borrowed books
9. Recommend books for a customer
10. Check late returns
11. Exit
Enter your choice: 1
Enter CSV file name: sample_books.csv

Library Management System
1. Load books from CSV
2. Add a book
3. Register a customer
4. Borrow a book
5. Return a book
6. Search for books
7. Display available books
8. Display customer borrowed books
9. Recommend books for a customer
10. Check late returns
11. Exit
Enter your choice: 7
1984 by George Orwell (1949, 1234567890, Dystopian, 5, [])
To Kill a Mockingbird by Harper Lee (1960, 1234567891, Fiction, 3, [])
The Great Gatsby by F. Scott Fitzgerald (1925, 1234567892, Classic, 4, [])
Moby Dick by Herman Melville (1851, 1234567893, Adventure, 2, [])
Pride and Prejudice by Jane Austen (1813, 1234567894, Romance, 6, [])

Library Management System
1. Load books 